<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Stochastic Approximation</div>

L'objectif de ce notebook est de vous donner l'occasion de comprendre plus en profondeur le principe de la Descente de Gradient Stochastique qui permet la mise à jour des poids d'un réseau de neurones. 

1. [Rappel : Descente de Gradient](#sec1)
2. [Exemple : réseau de neurones simplifié](#sec2)
3. [Approximation Stochastique](#sec3)
5. [Pour aller plus loin...](#sec4)

# <a id="sec1"></a>Rappels : Descente de Gradient

Commençons par quelques rappels sur le fonctionnement de la descente de gradient classique.

Tout d'abord, nous voulons trouver le minimum de la fonction suivante :

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def function(a):
    X = [-1, 0, 1]
    return sum([(2*np.sin(x)+1 - a[0] - a[1]*x)**2 for x in X])

def gradient(a):
    X = [-1, 0, 1]
    grad = np.zeros_like(a)
    grad[0] = sum([-2*(2*np.sin(x)+1-a[0]-a[1]*x) for x in X])
    grad[1] = sum([-2*(2*np.sin(x)+1-a[0]-a[1]*x)*x for x in X])
    return grad

def display_func(R, func):
    fig = plt.figure(figsize = (20,7))
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    x, y = np.linspace(-3, 3, 100), np.linspace(-3, 3, 100)
    z = np.array([[func([a, b]) for b in y] for a in x])
    x, y = np.meshgrid(x, y)
    ax1.plot_wireframe(x, y, z)
    ax1.set_xlabel("a[0]")
    ax1.set_ylabel("a[1]")
    ax1.set_zlabel("f(a)")

    if R != []:
        ax1.plot([r[0] for r in R], [r[1] for r in R], [func(r) for r in R], 'r-')

        ax2 = fig.add_subplot(1, 2, 2)
        ax2.plot([func(r) for r in R], 'g-')
        ax2.set_xlabel("Number of steps")
        ax2.set_ylabel("f(x*)")

    plt.show()

In [ ]:
display_func([], function)

Dans notre cas, une première possibilité serait de calculer le gradient de la fonction et de trouver analytiquement en quel point il s'annule afin d'obtenir une solution exacte du problème.

Malheureusement, même si nous savons calculer le gradient de la fonction en tout point, dans certains cas il est difficile voire impossible d'évaluer en quel point il s'annule. Nous allons donc procéder différement.

**Méthode de la descente de gradient à pas fixe :**

Soient $\mathbb{E} (\langle\cdot,\cdot\rangle)$ un espace hilbertien et $x\in\mathbb{E}\mapsto f(x)\in \mathbb{R}$ une fonction différentiable. On note  $\nabla f(x)$ le gradient de $f$ en $x$.

**Théorème | Algorithme du gradient** On se donne un point initial $x_0\in\mathbb{E}$ et un seuil de tolérance $\varepsilon > 0$. L'algorithme du gradient définit une suite $x_1, x_2, \ldots \in\mathbb{E}$, jusqu'à ce qu'un test d'arrêt soit satisfait. Il passe de $x_k$ à $x_{k+1}$ par les étapes suivantes :
- *Simulation* : calcul de $\nabla f(x_k)$.
- *Calcul du pas* $\alpha_k>0$ par une règle de recherche linéaire sur $f$ en $x_k$ le long de la direction $-\nabla f(x_k)$.
- *Nouvel itéré* : $x_{k+1} = x_k - \alpha_k \nabla f(x_k).$
- *Test d'arrêt*

Pour une fonction strictement convexe, il est possible de montrer la convergence de cet algorithme vers l'unique minimum $x^*$ de $f$ .


<div class="alert alert-warning"><b>Exercice : </b><br>
Calculer une approximation du minimum de la fonction précédente grâce à une decente de gradient à pas fixe. Quelles sont les différentes conditions d'arrêt possibles ?
</div>

In [ ]:
# %load solutions/code1.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

def fixed_step_descent(function, gradient, eps, step_size):
    x = [...]
    result = [x.copy()]  # At each step of the algorithm, add the value of x in result to get the display
    
    ## WRITE YOUR CODE HERE
    
    return result

result = fixed_step_descent(function, gradient, ... , ... )
print("Le minimum obtenu est : ", round(function(result[-1]),5), " pour a = ", result[-1])

display_func(result, function)

Malheureusement, cette méthode peut rencontrer de nombreux problèmes, par exemple lorsque le minimum se trouve au fond d'une vallée étroite, la convergence est parfois impossible. De plus, la recherche du pas optimal peut être longue. 

Il existe d'autres algorithmes qui offrent souvent de meilleurs résultats, comme par exemple :
- la **méthode de Newton** qui définit la suite des $x_k$ de la façon suivante : $x_{k+1} = x_k - \frac{f(x_k)}{\nabla f(x_k)}$
- la **méthode BFGS** qui consiste à calculer en chaque étape une matrice, qui multipliée au gradient permet d'obtenir une meilleure direction. De plus, cette méthode peut être combinée avec une méthode plus efficace de recherche linéaire afin d'obtenir la meilleure valeur de α. 

Rappelons toutefois que ces différentes méthodes fonctionnent bien pour des fonctions convexes.

In [ ]:
n_boulders = 150
boulders = np.random.uniform(0., 1., (n_boulders, 3))
boulders[:, 2] = 1/(3 ** np.random.choice(range(2, 4), (n_boulders)))

X, Y = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
Z = np.zeros(X.shape)

for i in range(n_boulders):
    eval_ = (X - boulders[i, 0]) ** 2 + (Y - boulders[i, 1]) ** 2 - boulders[i, 2] ** 2
    Z[eval_ < 0] += eval_[eval_<0]

fig = plt.figure(figsize=(5, 5))
plt.contour(X, Y, Z, alpha=.5)
plt.show()

Dans un cas non convexe comme le précédent, on pourrait appliquer plusieurs fois la méthode en partant de différents points de départ afin d'explorer tous les puits de potentiel. Il serait alors possible de trouver les différents points de départ par une méthode métaheuristique, comme un algorithme génétique ou un recuit simulé.

# <a id="sec2"></a>Exemple : réseau de neurones simplifié

Changeons un peu de contexte : nous nous plaçons maintenant dans le cas où nous voulons approximer une fonction inconnue par une fonction linéaire de type $y = a_0 + a_1*x$. Cependant, nous connaissons seulement la valeur de la fonction en quelques points $(x_i, y_i)$. Pire, ces échantillons sont bruités ! 

On pourra rapprocher cet exemple du cas de la **mise à jour des poids d'un réseau de neurones**. Ici on a un réseau hyper-simplifié, avec seulement un poids, $a_1$, et un biais, $a_0$, et à terme on veut qu'il puisse nous prédire la valeur de $y$ pour un échantillon $x$.

In [ ]:
def generate_samples(n):
    samples = []
    for i in range(n):
        x = random.uniform(-1, 1)
        samples.append([x, 2*np.sin(x)+1 + 0.1*np.random.randn()])
    return np.array(samples)

n = 100
samples = generate_samples(n)
X = samples[:,:-1]
X = np.c_[np.ones(n), X]    # add a column of ones at the beginning
Y = samples[:,-1]
print(X.shape, Y.shape)

plt.plot(samples[:,0], samples[:,1], 'b.')
plt.show()

Naïvement, notre première idée est de trouver $a_0$ et $a_1$ qui minimisent la somme des carrés des erreurs $y_i - (a_0 + a_1 * x_i)$ :

$$ \min_{(a_0, a_1) \in \mathbb{R}^2} \sum_{i=1}^{N} (y_i - a_0 - a_1 * x_i)^2 $$

La première solution possible consiste à calculer sous forme matricielle la solution exacte de ce problème grâce à la formule suivante : 

$$ \hat{a} = (X^{T}X)^{-1}X^{T}Y $$

<div class="alert alert-warning"><b>Exercice : </b> Appliquez cette formule pour trouver les $a_0$ et $a_1$ optimaux.
</div>

In [ ]:
# %load solutions/code2.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

a = ... ## WRITE YOUR CODE HERE

Bien que cette méthode nous fournisse la meilleure solution possible, elle nécessite d'inverser une matrice de taille N. Dans notre cas, nous avons peu d'échantillons mais avec un *dataset* de plusieurs millions d'entrées, une telle inversion nous coûterait cher en temps de calcul et en mémoire.

Une autre possibilité est donc de reprendre la méthode précédente de **descente de gradient**.

La fonction à minimiser est dorénavent la fonction de coût, ou *loss function* :
$$  L(a) = \sum_{i=1}^{N} (y_i - a_0 - a_1 * x_i)^2 $$

<div class="alert alert-warning"><b>Exercice : </b> Codez la <i>loss function</i>.
</div>

In [ ]:
# %load solutions/code3.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

def loss(a):
    return ## WRITE YOUR CODE HERE

display_func([], loss)

Maintenant que nous avons notre fonction à minimiser, nous pouvons à nouveau appliquer la méthode du gradient à pas fixe grâce à la même formule que précédemment : 
$$ a_{k+1} = a_k - \beta_k \nabla f(a_k) $$ 

<div class="alert alert-warning"><b>Exercice : </b> Calculez $\nabla L$ puis implémentez la descente de gradient à pas fixe.
</div>

In [ ]:
# %load solutions/code4.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

def loss_gradient(a,X,Y):
    grad = [0]*X.shape[1]
    # WRITE YOUR CODE HERE
    return grad

def gradient_descent(X,Y, eps, step_size):
    a = [0]*X.shape[1]
    result = [a.copy()]
    
    # WRITE YOUR CODE HERE
        
    return result

result = gradient_descent(X,Y, ... , ...)

print("Le minimum obtenu est : ", round(loss(result[-1]),5), " pour a = ", result[-1])

display_func(result, loss)


Très bien ! Cette méthode nous a donc permis d'obtenir une bonne approximation du minimum de notre *loss function*. Mais est-ce que vous ne flairez pas une arnaque ? C'était un peu trop simple, non ?

Le problème ici, c'est que l'objectif d'un réseau de neurones, ce n'est pas simplement de faire une régression linéaire entre des points, mais de pouvoir reproduire la fonction qui est à l'origine de ces points, sinon on dit qu'on *overfit* le dataset. Le *dataset* que nous utilisons étant un nombre fini de points bruités, il nous permet seulement d'obtenir une **approximation** de la *loss function*, et donc en minimisant celle-ci avec la descente de gradient classique, nous ne sommes pas sûrs de converger vers la vraie fonction que nous recherchons.

---------------------------------

Réécrivons tout cela de façon un peu plus mathématique :

Nous avons un dataset de points $(x, y)$ tirés d'une distribution de probabilité $p(x, y)$. Apprendre un réseau de neurones qui prédit correctement $y$ en fonction de $x$ correspond à trouver les paramètres $\theta$ qui minimisent la fonction suivante :

$$L(\theta) = \displaystyle \mathbb{E}_{(x,y)\sim p(x,y)} \left[ \left(f_\theta(x) - y\right)^2 \right] = \int_{x,y} \left[ \left(f_\theta(x) - y\right)^2 \right] \mathrm{d}p(x,y) $$

Ceci est la vraie loss function que nous voulons minimiser. La descente de gradient classique nous dit qu'il faut se déplacer dans la direction opposée au gradient de $L(\theta)$ pour trouver $\theta$ optimal. Ecrivons ce gradient :

\begin{align*}
\displaystyle \nabla_\theta L(\theta) &= \nabla_\theta \left[ \mathbb{E}_{(x,y)\sim p(x,y)} \left[ \left(f_\theta(x) - y\right)^2 \right] \right]\\
&= \mathbb{E}_{(x,y)\sim p(x,y)} \left[ \nabla_\theta \left[ \left(f_\theta(x) - y\right)^2 \right] \right]\\
&= \mathbb{E}_{(x,y)\sim p(x,y)} \left[ 2 \left(f_\theta(x) - y\right) \nabla_\theta f_\theta(x) \right]
\end{align*}

Donc, le gradient de $L(\theta)$ est l'espérance de $2 \left(f_\theta(x) - y\right) \nabla_\theta f_\theta(x)$. En d'autres mots :

$$\nabla_\theta L(\theta) = \int_{x,y} 2 \left(f_\theta(x) - y\right) \nabla_\theta f_\theta(x) \mathrm{d}p(x,y)$$

Le problème avec cette expression est que nous avons besoin de connaître $p(x,y)$ pour toutes les paires $(x,y)$ possibles. Pour cela il nous faudrait une quantité infinie de donnée. On peut cenpendant approximer ce gradient avec un dataset fini $\left\{\left(x_i,y_i\right)\right\}_{i\in [1,N]}$:
$$\nabla_\theta L(\theta) \approx \sum_{i=1}^N 2 \left(f_\theta(x_i) - y_i\right) \nabla_\theta f_\theta(x_i)$$

Ici nous avons une *estimation bruitée du gradient* , qui converge vers le vrai gradient lorsque le nombre d'échantillons tend vers l'infini. C'est cette approximation que nous avons utilisé plus tôt avec la descente de gradient classique. Toutefois, même si cette approximation du gradient converge vers le vrai gradient pour un nombre d'échantillons infini, nous n'avons aucune preuve qu'avec cette approximation la descente de gradient permettra à notre réseau de converger vers la vraie fonction.

Pour résoudre ce problème, nous avons besoin d'un outil supplémentaire : l'**approximation stochastique** !

# <a id="sec3"></a>Approximation Stochastique

## L'algorithme de Robbins-Monro

La méthode de l'approximation stochastique fut inventée par Herbert Robbins et Sutton Monro et publiée en 1951 dans l'article fondateur [*A Stochastic Approximation Method*](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586)

**Descriptions de l'algorithme :**

Soit $M$ une fonction et $\alpha$ une constante telle que l'équation $M(\theta) = \alpha$ possède une unique racine en $\theta^*$.
On suppose que l'on ne peut pas observer directement les valeurs de la fonction $M$, mais que nous pouvons cependant obtenir des mesures de la variable aléatoire $N(\theta)$ telle que $E[N(\theta)] = M(\theta)$. On veut estimer $\theta^*$ en faisant des observation successives de $N$ en $\theta_1$, $\theta_2$ , $\theta_3$, telles que la suite des $\theta_n$ soit de la forme :
$$ \theta_{n+1} = \theta_{n} - a_{n} (N(\theta_{n}) - \alpha) $$

où $(a_{n})$ est une suite de pas positifs. 


**Hypothèses de convergence :**

Robbins et Monro ont montré que la suite des $\theta_n$ converge en $L^2$ vers $\theta^*$ si les conditions suivantes sont remplies :
* $N(\theta)$ est bornée
* $M$ est croissante
* $M'(\theta^*)$ existe et est positive
* la suite des $a_n$ vérifie $$\qquad \sum^{\infty}_{n=0}a_n = \infty \quad \mbox{ and } \quad \sum^{\infty}_{n=0}a^2_n < \infty \quad $$

Une suite particulière a été proposée par Robbins et Monro : $a_n = \frac{a}{n}$ avec $a > 0$.

**Explications :**

La preuve formelle de la convergence se trouve dans l'article [*A Stochastic Approximation Method*](https://projecteuclid.org/download/pdf_1/euclid.aoms/1177729586), mais voici quelques explications intuitives sur les conditions imposées par l'algorithme :
* Le fait que $N(\theta)$ soit bornée impose que le bruit sur les échantillons soit borné, donc non biaisé et identique sur l'ensemble des $x_i$.
* Les conditions 2 et 3 imposent que la fonction $M$ ait un comportement "raisonnable" près de sa racine et à l'infini.
* $\sum^{\infty}_{n=0}a_n = \infty$ nous dit que quelque soit le point de départ $\theta_0$, même s'il est loin du minimum, l'algorithme pourra atteindre $\theta^*$.
* $\sum^{\infty}_{n=0}a^2_n < \infty$ oblige le pas de l'algorithme à décroître et évite les oscillations autour du minimum.

## Application à l'optimisation

La théorie de la *descente de gradient stochastique* nous dit donc que si nous avons $g(\theta)$, un estimateur bruité de $\nabla_\theta L(\theta)$, alors la suite $\theta_k$ converge vers un minimum local de $L(\theta)$, pour
$$\theta_{k+1} = \theta_k - \alpha_k g(\theta_k)$$
avec les conditions  $\sum \alpha_k = \infty$ et $\sum \alpha_k^2 < \infty$ (appelées conditions de Robbins-Monro).

---------------------

Pour revenir à la mise à jour des poids d'un réseau de neurones, nous avons donc $g(\theta)$, une fonction localement convexe qui approxime le gradient $\nabla_\theta L(\theta)$ :
$$g(\theta) = \sum_{i=1}^N 2 \left(f_\theta(x_i) - y_i\right) \nabla_\theta f_\theta(x_i)$$
Pour appliquer l'algorithme nous allons utiliser la suite conseillée par Robbins et Monro : $\alpha_n = \frac{\alpha}{n}$ avec $\alpha > 0$.

$\alpha$ est aussi appelé le *learning rate*.

<div class="alert alert-warning"><b>Exercice : </b> Reprenez les fonctions précédentes pour $L(\theta)$ et son gradient et appliquez la méthode de l'approximation stochastique. Attention, l'algorithme est très sensible au choix de $\alpha$.
</div>

In [ ]:
# %load solutions/code5.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

def stochastic_gradient_descent(X,Y,lr,eps):
    a = [0]*X.shape[1]
    result = [a.copy()]
    
    # WRITE YOUR CODE HERE
        
    return result

result = stochastic_gradient_descent(X,Y, ... , ...)

print("Le minimum obtenu est : ", round(loss(result[-1]),5), " pour a = ", result[-1])

display_func(result, loss)

Et voilà ! Nous avons donc une belle descente de gradient stochastique fonctionnelle !
Il ne nous manque plus qu'une dernière astuce pour améliorer les performances de l'algorithme et nous aurons retrouvé l'algorithme qui est aujourd'hui couramment utilisé pour la mise à jour des poids d'un réseau de neurones.

## Mini-Batch Gradient Descent

L'algorithme de Robbins-Monro nous impose peu de conditions sur l'estimateur de $L(\theta)$. Jusqu'à présent nous l'avons calculé de la façon suivante : 
$$g(\theta) = \sum_{i=1}^N 2 \left(f_\theta(x_i) - y_i\right) \nabla_\theta f_\theta(x_i)$$

Mais rien ne nous oblige à sommer sur l'ensemble du dataset ! Si on ne calcule la *loss* que sur un sous-ensemble aléatoire de taille $n$ du dataset nous aurons toujours un estimateur convenable de la *loss*, même s'il sera alors plus bruité. Cette astuce nous permettra surtout d'économiser beaucoup de temps de calcul lors de l'évaluation de $g(\theta)$. 

Lorsque $n=1$, l'estimation du gradient est basée sur un seul échantillon, elle est donc très bruitée, et la convergence peut être lente et instable. Lorsque $n \rightarrow N$, le niveau de bruit décroît mais le temps de calcul est plus long. En pratique, une valeur de $n$ entre 32 et 1024 fonctionne assez bien pour de grands *datasets*. (Généralement, on aime bien utiliser des puissances de 2.) 

On appelle un tel sous-ensemble du *dataset* un *minibatch*. Pendant l'execution de l'algorithme, nous allons donc découper le *dataset* en *minibatch* et le parcourir plusieurs fois. Un parcours entier du *dataset* est nommé une *epoch*.

<div class="alert alert-warning"><b>Exercice : </b> Implémentez la <i>minibatch gradient descent</i>.
</div>

In [ ]:
# %load solutions/code6.py
# If you get stuck, uncomment the line above to load a correction in this cell (then you can execute this code).

def mini_batch_SGD(X,Y,lr, nb_epoch, batch_size):
    a = [0]*X.shape[1]
    result = [a.copy()]
    
    # WRITE YOUR CODE HERE
    
    return result

result = mini_batch_SGD(X,Y, ... , ... , ...)

print("Le minimum obtenu est : ", round(loss(result[-1]),5), " pour a = ", result[-1])

display_func(result, loss)

# Pour aller plus loin...

De nombreuses améliorations sur l'algorithme SGD initial ont été proposées et utilisées.
En particulier, la nécessité de fixer un taux d'apprentissage a souvent été décrite comme problématique. Un choix de paramètre trop grand peut rendre l'algorithme divergent tandis qu'un choix trop petit rend la convergence trop lente.

## Moyennage

En pratique, l'algorithme de Robbins-Monro est très sensible aux paramètres comme le point de départ $\theta_0$ ou la calibration de la suite de pas $(\alpha_n)$. La vitesse optimale de l'algorithme est atteinte pour un pas de la forme $\alpha_n=\alpha/n$ mais le choix de la constante $\alpha$ est extrêmement important et la déterminer en pratique s'avère délicat. 

Pour contourner ce choix, Polyak et Juditsky proposèrent en 1992 dans [**Acceleration of Stochastic Approximation**](http://www.meyn.ece.ufl.edu/archive/spm_files/Courses/ECE555-2011/555media/poljud92.pdf) une version «moyennisée» de cet algorithme en prenant la suite de pas $(\alpha_n)$ à décroissance plus lente que la vitesse «optimale», par exemple $\alpha_n=n^{-3/4}$. Ils lissent ensuite la suite d'estimateurs $\theta_{n+1} - \theta_n = a_n(\alpha - N(\theta_n))$ en considérant le «moyennisé» : $$\bar{\theta}_n = \frac{1}{n} \sum^{n-1}_{i=0} \theta_i$$ 
On peut alors montrer que $\bar{\theta}_n$ converge vers l'unique racine $\theta^*$. En pratique, cet algorithme est beaucoup moins sensible au choix des paramètres.

## SGD Moment

La *méthode du moment* apparaît dans l'article [**Learning representations by back-propagating errors**](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf) de Rumelhart, Hinton et Williams en 1986 sur l'apprentissage par rétro-propagation.
La méthode SGD avec moment conserve en mémoire la mise-à-jour à chaque étape ($\Delta \theta$), et calcule la suivante comme une combinaison convexe du gradient actuel et de la modification précédente :

$$\Delta \theta = \eta \nabla g(\theta) + \alpha \Delta \theta$$
$$\theta = \theta - \eta \Delta \theta $$

Le nom *moment* vient d'une analogie avec le moment en physique : le vecteur de paramètres $\theta$, considéré comme une particule qui voyage au travers de l'espace des paramètres (souvent en grande dimension), subit une accélération via le gradient (qui agit comme une force physique). Contrairement à la méthode SGD classique, cette variante a tendance à continuer de *voyager* dans la même direction, en empêchant les oscillations.

## AdaGrad

*AdaGrad* (ou *Adaptive Gradient Algorithm*) est une amélioration de la méthode SGD qui va déterminer automatiquement un taux d'apprentissage pour chaque paramètre. Cette méthode a été présentée en 2011 dans [**Adaptive Subgradient Methods for Online Learning and Stochastic Optimization**](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf). On doit toujours choisir un taux d'apprentissage commun ($\eta$), mais celui-ci est multiplié par les éléments du vecteur $G_{j,j}$, qui est obtenu comme diagonale de la matrice G suivante :

$$G = \sum_{\tau=1}^t g_\tau g_\tau^\mathsf{T}$$

où $g_\tau = \nabla Q_i(\theta)$ est le gradient à l'étape $\tau$.
La diagonale est donnée par : $G_{j,j} = \sum_{\tau=1}^t g_{\tau,j}^2$.

Ce vecteur est mis-à-jour à chaque itération.
Ainsi, la formule pour chaque mise-à-jour pour chaque paramètre est désormais :

$$ \theta_j = \theta_j - \frac{\eta}{\sqrt{G_{j,j}}} g_j$$

Chaque $G_{j,j}$ donne un facteur multiplicatif appliqué au taux d'apprentissage correspondant à l'unique paramètre $\theta_i$.
Et comme le dénominateur de ce facteur ($\sqrt{G_i} = \sqrt{\sum_{\tau=1}^t g_\tau^2}$) est la norme euclidienne usuelle des dérivées précédentes, les mises-à-jour trop importantes des paramètres sont atténuées tandis que les petites modifications sont faites avec un taux d'apprentissage plus grand (et donc sont amplifiées).

## Algorithme de Kiefer-Wolfowitz

L'algorithme de Kiefer-Wolfowitz a été introduit en 1952 dans [**Stochastic estimation of the maximum of a regression function**](https://projecteuclid.org/euclid.aoms/1177729392) pour estimer le maximum d'une fonction $f:x↦E[F(x,ξ)]$
, connue à une perturbation près. Lorsque le gradient de $f$ n'est pas observable, cet algorithme permet d'approcher le maximum en utilisant une approximation du gradient de $f$ par des différences finies. Il est défini pour $n≥1$, par l'itération :

$$X_{n+1}=X_n−γ_{n+1} \frac{F(X_n+c_n,ξ^1_n)−F(x_n−c_n,ξ^2_n)}{2c_n}$$

où $(ξ^1_n)_n$ et $(ξ^2_n)_n$ sont deux suites de variables aléatoires indépendantes et identiquement distribuées, $(c_n)$ et $(γ_n)$ sont deux suites déterministes positives et décroissantes vers 0 vérifiant:

$$∑nγ_n=∞, ∑nc_nγ_n<∞, ∑n(γ_n/c_n)^2<∞$$

La suite $(c_n)$ désigne les largeurs des différences finies utilisées pour l'approximation du gradient, tandis que la suite $(γ_n)$ représente les pas de la descente du gradient. Sous de bonnes hypothèses de régularité et convexité/concavité de la fonction $f$, cet algorithme converge vers le maximum/minimum visé.

# Liens et Références

* [Wikipédia - Stochastic Approximation](https://en.wikipedia.org/wiki/Stochastic_approximation)
* [Optimization Methods for Large-Scale Machine Learning](https://arxiv.org/pdf/1606.04838.pdf)
* [Introduction to Stochastic Approximation Algorithms](http://www.professeurs.polymtl.ca/jerome.le-ny/teaching/DP_fall09/notes/lec11_SA.pdf)
* [Wikipédia - Stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
* [Algorithmes Stochastiques](http://cenac.perso.math.cnrs.fr/hdr/algo-stochastiques.html)